In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_56569/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [5]:
analize = Analizer(0.9)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_8_8_19,0.999948,0.786449,0.999861,0.999966,0.999923,0.000074,0.303104,0.000373,0.000087,0.000230,0.028382,0.008582,1.000001,0.008947,3537.032522,5681.035098,"Hidden Size=[40, 39], regularizer=0.5, learnin..."
1,model_8_8_22,0.999948,0.786449,0.999861,0.999966,0.999923,0.000074,0.303104,0.000373,0.000087,0.000230,0.028382,0.008582,1.000001,0.008947,3537.032515,5681.035091,"Hidden Size=[40, 39], regularizer=0.5, learnin..."
2,model_8_8_23,0.999948,0.786449,0.999861,0.999966,0.999923,0.000074,0.303104,0.000373,0.000087,0.000230,0.028383,0.008582,1.000001,0.008947,3537.032501,5681.035076,"Hidden Size=[40, 39], regularizer=0.5, learnin..."
3,model_8_8_20,0.999948,0.786449,0.999861,0.999966,0.999923,0.000074,0.303104,0.000373,0.000087,0.000230,0.028382,0.008582,1.000001,0.008947,3537.032499,5681.035075,"Hidden Size=[40, 39], regularizer=0.5, learnin..."
4,model_8_8_24,0.999948,0.786449,0.999861,0.999966,0.999923,0.000074,0.303104,0.000373,0.000087,0.000230,0.028383,0.008582,1.000001,0.008947,3537.032477,5681.035053,"Hidden Size=[40, 39], regularizer=0.5, learnin..."
5,model_8_8_21,0.999948,0.786449,0.999861,0.999966,0.999923,0.000074,0.303104,0.000373,0.000087,0.000230,0.028385,0.008582,1.000001,0.008947,3537.032454,5681.035030,"Hidden Size=[40, 39], regularizer=0.5, learnin..."
6,model_8_8_18,0.999948,0.786449,0.999861,0.999966,0.999923,0.000074,0.303104,0.000373,0.000087,0.000230,0.028381,0.008582,1.000001,0.008947,3537.032422,5681.034998,"Hidden Size=[40, 39], regularizer=0.5, learnin..."
7,model_8_8_15,0.999948,0.786449,0.999861,0.999966,0.999923,0.000074,0.303104,0.000373,0.000087,0.000230,0.028381,0.008582,1.000001,0.008947,3537.032396,5681.034972,"Hidden Size=[40, 39], regularizer=0.5, learnin..."
8,model_8_8_16,0.999948,0.786449,0.999861,0.999966,0.999923,0.000074,0.303104,0.000373,0.000087,0.000230,0.028382,0.008582,1.000001,0.008947,3537.032383,5681.034959,"Hidden Size=[40, 39], regularizer=0.5, learnin..."
9,model_8_8_14,0.999948,0.786449,0.999861,0.999966,0.999923,0.000074,0.303104,0.000373,0.000087,0.000230,0.028378,0.008582,1.000001,0.008947,3537.032371,5681.034946,"Hidden Size=[40, 39], regularizer=0.5, learnin..."
